In [2]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime

## Group Assignment
### Team Number: XX
### Team Member Names:
### Team Strategy Chosen: __________(Market Beat or Market Meet)

Disclose any use of AI for this assignment below (detail where and how you used it).  Please see the course outline for acceptable uses of AI.


In [3]:
tickers_file_name = "Tickers_Example.csv"
tickers_file_original = pd.read_csv(tickers_file_name, header = None, names=['Ticker'])

start_date = "2023-10-02"
end_date = "2024-09-30"
drop_tickers = []

for tick in tickers_file_original['Ticker']:    

    try:

        ticker = yf.Ticker(tick) 

        if ticker.fast_info['quoteType'] != 'EQUITY':
            drop_tickers.append(tick)
            continue
                
        if ticker.fast_info.get('currency') not in ['USD', 'CAD']:
            drop_tickers.append(tick)
            continue

        history = ticker.history(start = start_date, end = end_date)

        trading_days = history['Volume'].resample('ME').count()
        volume = history['Volume'].resample('ME').sum()
        trading_days = trading_days[trading_days>=18]
        volume = volume[volume.index==trading_days.index]
        monthly_volume = volume.mean()

        if monthly_volume<100000: 
            drop_tickers.append(tick)
        
    except:
            drop_tickers.append(tick)
            pass
    
tickers_file_original.drop(tickers_file_original[tickers_file_original["Ticker"].isin(drop_tickers)].index, inplace=True)
tickers_file_original = tickers_file_original.reset_index(drop=True)
# print(tickers_file_original)

$AGN: possibly delisted; no price data found  (period=5d) (Yahoo error = "No data found, symbol may be delisted")
$CELG: possibly delisted; no price data found  (period=5d) (Yahoo error = "No data found, symbol may be delisted")
$MON: possibly delisted; no price data found  (period=5d) (Yahoo error = "No data found, symbol may be delisted")
$RTN: possibly delisted; no price data found  (period=5d) (Yahoo error = "No data found, symbol may be delisted")


In [10]:
sp500 = yf.Ticker("^GSPC").history(start=start_date, end=end_date)['Close']
sp500.index = sp500.index.date
tsx60 = yf.Ticker("XIU.TO").history(start=start_date, end=end_date)['Close']
tsx60.index = tsx60.index.date

total_investment = 1000000
sp500val = (total_investment / sp500.iloc[0]) * sp500.iloc[-1]
sp500val = round(sp500val,2)
# sp500val

tsx60val = (total_investment / tsx60.iloc[0]) * tsx60.iloc[-1]
tsx60val = round(tsx60val,2)
# tsx60val

average = (sp500val + tsx60val) / 2
average = round(average,2)
average

1311957.23

In [ ]:
stocks = {key: 0 for key in tickers_file_original['Ticker'].values}
current_data = pd.DataFrame()
metrics = pd.DataFrame(index=list(stocks.keys()), columns=['SP-beta','SP-corr','TSX-beta','TSX-corr'])

for i in stocks:
    current_data[i] = yf.Ticker(i).history(start=start_date, end=end_date)["Close"]
    
    
    covariance = sp500.cov(current_data[i])
    variance = sp500.var()
    beta = covariance / variance

    correlation = sp500.corr(current_data[i])
    metrics.loc[i,'SP-corr'] = correlation
    metrics.loc[i,'SP-beta'] = beta

    covariance2 = tsx60.cov(current_data[i])
    variance2 = tsx60.var()
    beta2 = covariance / variance

    correlation2 = tsx60.corr(current_data[i])
    metrics.loc[i,'TSX-corr'] = correlation2
    metrics.loc[i,'TSX-beta'] = beta2

    stocks[i] = round(100 * (correlation + beta + beta2 + correlation2),2)


metrics
    



,SP-beta,SP-corr,TSX-beta,TSX-corr
AAPL,0.031744,0.678171,0.031744,0.626303
ABBV,0.035126,0.851557,0.035126,0.879057
ABT,0.009455,0.570702,0.009455,0.637014
ACN,0.00463,0.076545,0.00463,0.160075
AIG,0.01192,0.88176,0.01192,0.860027
AMZN,0.042903,0.924021,0.042903,0.865154
AXP,0.081293,0.970728,0.081293,0.971736
BA,-0.031279,-0.528617,-0.031279,-0.530535
BAC,0.010859,0.96535,0.010859,0.929716
BB.TO,-0.001521,-0.86532,-0.001521,-0.811991


In [26]:
stocks = {key: 0 for key in tickers_file_original['Ticker'].values}
current_data = pd.DataFrame()
metrics = pd.DataFrame(index=list(stocks.keys()), columns=['SP-beta','SP-corr','TSX-beta','TSX-corr'])

sp500_returns = sp500.pct_change(fill_method=None)
tsx60_returns = tsx60.pct_change(fill_method=None)

for i in stocks:
    current_data[i] = yf.Ticker(i).history(start=start_date, end=end_date)["Close"]
    stock_returns = current_data[i].pct_change().dropna()
    stock_returns.index = stock_returns.index.date
    
    common_dates = stock_returns.index.intersection(sp500_returns.index).intersection(tsx60_returns.index)

    stock_returns = stock_returns.loc[common_dates]
    aligned_sp500_returns = sp500_returns.loc[common_dates]
    aligned_tsx60_returns = tsx60_returns.loc[common_dates]
    
    covariance = np.cov(stock_returns, aligned_sp500_returns)[0, 1]
    variance = np.var(aligned_sp500_returns)
    beta_sp500 = covariance / variance
    
    correlation_sp500 = np.corrcoef(stock_returns, aligned_sp500_returns)[0, 1]
    
    covariance_tsx = np.cov(stock_returns, aligned_tsx60_returns)[0, 1]
    variance_tsx = np.var(aligned_tsx60_returns)
    beta_tsx60 = covariance_tsx / variance_tsx
    
    correlation_tsx60 = np.corrcoef(stock_returns, aligned_tsx60_returns)[0, 1]
    
    metrics.loc[i, 'SP-corr'] = correlation_sp500
    metrics.loc[i, 'SP-beta'] = beta_sp500
    metrics.loc[i, 'TSX-corr'] = correlation_tsx60
    metrics.loc[i, 'TSX-beta'] = beta_tsx60

    # stocks[i] = round(100 * (correlation_sp500 + beta_sp500 + beta_tsx60 + correlation_tsx60),2)


metrics

C:\Users\abbas\AppData\Local\Temp\ipykernel_12792\2970812930.py:10: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  stock_returns = current_data[i].pct_change().dropna()
C:\Users\abbas\AppData\Local\Temp\ipykernel_12792\2970812930.py:10: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  stock_returns = current_data[i].pct_change().dropna()
C:\Users\abbas\AppData\Local\Temp\ipykernel_12792\2970812930.py:10: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or speci

,SP-beta,SP-corr,TSX-beta,TSX-corr
AAPL,0.947254,0.534552,0.465565,0.235359
ABBV,0.203729,0.132373,0.234928,0.136743
ABT,0.114747,0.075521,0.216023,0.127366
ACN,0.715192,0.381639,0.665242,0.318007
AIG,0.743347,0.456687,0.899799,0.495221
AMZN,1.515451,0.664044,0.936852,0.36775
AXP,1.021988,0.537695,1.148115,0.54113
BA,0.907224,0.334471,1.079021,0.35637
BAC,0.859954,0.442248,1.31985,0.608053
BB.TO,1.35957,0.307291,1.455337,0.294671


In [ ]:
beta_threshold = 0.1  
penalty_weight = 50   

ranked_stocks = {}
for ticker, score in stocks.items():
    sp_beta = metrics.loc[ticker, 'SP-beta']
    tsx_beta = metrics.loc[ticker, 'TSX-beta']
    
    sp_penalty = abs(sp_beta - 1) * penalty_weight
    tsx_penalty = abs(tsx_beta - 1) * penalty_weight

    adjusted_score = score - (sp_penalty + tsx_penalty)
    ranked_stocks[ticker] = adjusted_score

ranked_stocks = dict(sorted(ranked_stocks.items(), key=lambda item: item[1], reverse=True))

stocks = dict(list(ranked_stocks.items())[:24])

stocks

{'BLK': -4.291685347265561,
 'AXP': -8.505167516753854,
 'BA': -8.589869043068315,
 'UPS': -12.796122687980438,
 'C': -15.698075167064072,
 'AIG': -17.842692193551898,
 'BK': -17.936580916159958,
 'BAC': -22.994813017600478,
 'BIIB': -24.637400576578756,
 'LLY': -27.965054618379025,
 'AMZN': -28.9299748730578,
 'AAPL': -29.359021258016995,
 'TXN': -30.066219088614144,
 'ACN': -30.9783040182638,
 'USB': -32.48105720083506,
 'RY.TO': -32.543746752619526,
 'TD.TO': -33.44055181837034,
 'CAT': -34.02072035296773,
 'UNP': -34.420380095060565,
 'PYPL': -39.7073423946248,
 'BB.TO': -40.74537614587176,
 'BMY': -59.760664634321344,
 'T.TO': -61.73927016439145,
 'PFE': -65.70338238015512}

In [29]:
!pip install cvxpy

   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ------- -------------------------------- 0.2/1.1 MB 2.5 MB/s eta 0:00:01
   ------------------- -------------------- 0.5/1.1 MB 4.2 MB/s eta 0:00:01
   ----------------------------------- ---- 1.0/1.1 MB 5.5 MB/s eta 0:00:01
   ---------------------------------------  1.1/1.1 MB 5.7 MB/s eta 0:00:01
   ---------------------------------------- 1.1/1.1 MB 4.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/736.4 kB ? eta -:--:--
   ----------------------- ---------------- 440.3/736.4 kB 9.2 MB/s eta 0:00:01
   ---------------------------------------- 736.4/736.4 kB 9.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/293.1 kB ? eta -:--:--
   --------------------------------------- 293.1/293.1 kB 17.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/8.4 MB ? eta -:--:--
   -- -------------------------

In [42]:
import cvxpy as cp
def give_weight(num_stocks):
    stocks_now = list(stocks.keys())[:num_stocks]
    prices = np.array(current_data[stocks_now].iloc[0])
    last_values = np.array(current_data[stocks_now].iloc[-1])
    w = cp.Variable(num_stocks)
    my_value = cp.sum(cp.multiply(last_values, w * 1000000 / prices))
    min_weight = (100 / (2 * num_stocks)) / 100
    objective = cp.Minimize(my_value - sp500val)
    constraints = [
        cp.sum(w) == 1,           
        w >= min_weight,                    
        w <= 0.15                      
    ]
    problem = cp.Problem(objective, constraints)
    problem.solve()
    return w.value


In [43]:
diff = sp500val
close = 0
for i in range(12,25):
    weight_now = give_weight(i)
    last_row = current_data[list(stocks.keys())[:i]].iloc[-1]
    first_row = current_data[list(stocks.keys())[:i]].iloc[0]
    curr_val = np.sum(np.multiply(last_row,np.divide(weight_now*1000000,first_row)))
    if (abs(curr_val - sp500val) < diff):
        diff = abs(curr_val - sp500val)
        close = curr_val
print(close)
print(diff)

1177361.3246391076
160709.54536089255


In [44]:
print(diff/close)

0.13649976604264139


In [45]:
sp500val

1338070.87

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.